## Piping a Prompt, Model, and an Output Parser

In [2]:
import os
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import CommaSeparatedListOutputParser

In [3]:
os.environ.setdefault("no_proxy", "127.0.0.1,localhost")
os.environ.setdefault("HTTPX_NO_PROXY", "127.0.0.1,localhost")
OLLAMA_URL = os.getenv("OLLAMA_URL", "http://127.0.0.1:11434")

model_name = "gemma3" #"llama3.2:3b"

max_tokens = 100
llm = ChatOllama(model=model_name, num_predict=max_tokens, temperature=0)

In [4]:
list_instructions = CommaSeparatedListOutputParser().get_format_instructions()
list_instructions

'Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`'

In [5]:
chat_template = ChatPromptTemplate.from_messages([
    ('human', 
     "I've recently adopted a {pet}. Could you suggest three {pet} names? \n" + list_instructions)])

print(chat_template.messages[0].prompt.template)

I've recently adopted a {pet}. Could you suggest three {pet} names? 
Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`


In [8]:
list_output_parser = CommaSeparatedListOutputParser()
chat_template_result = chat_template.invoke({'pet':'dog'})

chat_result = llm.invoke(chat_template_result)
chat_result

AIMessage(content='Finn, Jasper, Luna', additional_kwargs={}, response_metadata={'model': 'gemma3', 'created_at': '2025-11-12T07:45:28.3803305Z', 'done': True, 'done_reason': 'stop', 'total_duration': 981071300, 'load_duration': 309120800, 'prompt_eval_count': 54, 'prompt_eval_duration': 104119600, 'eval_count': 6, 'eval_duration': 563403800, 'model_name': 'gemma3'}, id='run--16fb7f38-d358-4aed-a4e9-130dafdfecf3-0', usage_metadata={'input_tokens': 54, 'output_tokens': 6, 'total_tokens': 60})

In [9]:
list_output_parser.invoke(chat_result)

['Finn', 'Jasper', 'Luna']

In [10]:
chain = chat_template | llm | list_output_parser

In [11]:
chain.invoke({'pet':'dog'})

['Finn', 'Jasper', 'Luna']